In [1]:
%cd /home/ducpham/Documents/LightRAG
# move to project

/home/ducpham/Documents/LightRAG


In [2]:
# init global config for classes 

from config import ConfigParams
from dataclasses import asdict

config = asdict(ConfigParams())
for k, v in config.items():
    print(f"Key: {k}  -> Value: {v}")

Key: log_level  -> Value: 0
Key: storage_classes  -> Value: {'JsonKVStorage': <class 'lightrag.kvstore.json_kv.JsonKVStorage'>, 'NanoVectorDBStorage': <class 'lightrag.vectorstore.nano.NanoVectorStorage'>, 'NetworkXStorage': <class 'lightrag.graphstore.networkx.NetworkXStorage'>}
Key: working_dir  -> Value: ./lightrag_cache_2024-12-19-12:59:49
Key: chunk_token_size  -> Value: 1200
Key: chunk_overlap_token_size  -> Value: 200
Key: tiktoken_model_name  -> Value: gpt-4o-mini
Key: entity_extract_max_gleaning  -> Value: 1
Key: entity_summary_to_max_token  -> Value: 500
Key: node_embedding_algorithm  -> Value: node2vec
Key: node2vec_params  -> Value: {'dimentions': 1536, 'num_walks': 10, 'walk_length': 10, 'windown_size': 2, 'iterations': 3, 'random_seed': 3}
Key: embedding_func  -> Value: {'embedding_dim': 1536, 'max_token': 8192, 'func': <function openai_embedding at 0x7aeaf22e53a0>, 'concurrent_limit': 16}
Key: embedding_batch_num  -> Value: 32
Key: embedding_func_max_async  -> Value: 16


In [28]:
import json

with open("./testing/data/entity.json", mode='r') as f:
    all_entities_data = json.load(f)

In [29]:
all_entities_data[0]

{'node_id': 'Eren',
 'node_data': {'entity_type': 'person',
  'entity_desc': 'Eren Yeager là nhân vật chính, có khả năng biến thành Titan',
  'entity_source_id': '1'}}

In [21]:
from lightrag.vectorstore import NanoVectorStorage
from lightrag.embeddings import openai_embedding
from lightrag.utils import compute_mdhash_id
from dataclasses import field


entity_vdb = NanoVectorStorage(
    namespace="Test_LightRAG",
    global_config=config,
    embedding_func=openai_embedding,
    meta_fields={"entity_name"}
)


if entity_vdb is not None: # for entity
        data_for_vdb = {
            compute_mdhash_id(content=dp['entity_name'], prefix="ent-"): {
                "entity_name": dp['entity_name'],
                "content": dp['entity_name'] + dp['entity_desc']
            }
            for dp in all_entities_data
        }
        await entity_vdb.upsert(data=data_for_vdb)



INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './lightrag_cache_2024-12-19-12:59:49/vdb_Test_LightRAG.db'} 0 data
